In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
import os
import mne
import torch.nn.functional as F
import datetime

In [5]:
# Load the data
data_path = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\bcic_iv_2a\processed\2024-06-10_22-20-55\dataset_2024-06-10_22-20-55.pt"
label_path = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\bcic_iv_2a\processed\2024-06-10_22-20-55\labels_2024-06-10_22-20-55.pt"

data = torch.load(data_path)
labels = torch.load(label_path)


# Create a dataset and dataloaders
dataset = TensorDataset(data, labels)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [6]:
class Advanced_EEG_MI_CNN(nn.Module):
    def __init__(self):
        super(Advanced_EEG_MI_CNN, self).__init__()
        # First convolutional layer with Batch Normalization and ReLU activation
        self.conv1 = nn.Conv2d(in_channels=25, out_channels=32, kernel_size=(3, 3), padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # Second convolutional layer with Batch Normalization and ReLU activation
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # Third convolutional layer with Batch Normalization and ReLU activation
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(p=0.5)
        
        # Calculate the size of the input to the first fully connected layer
        # After three pooling layers, the spatial dimensions are reduced by a factor of 8
        # Initial scales dimension: 43 -> After pooling: 43 / 8 = 5.375 -> rounded down to 5
        final_scale_dim = 5
        final_time_dim = 15  # Assuming time dimension after pooling layers is 15, adjust if needed
        self.fc1 = nn.Linear(128 * final_scale_dim * final_time_dim, 256)
        
        # Fully connected layers
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 2)  # Assuming 2 classes for binary classification

    def _forward_conv_layers(self, x):
        # Forward pass through the first convolutional layer
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        # Forward pass through the second convolutional layer
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        # Forward pass through the third convolutional layer
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        return x

    def forward(self, x):
        # Forward pass through the convolutional layers
        x = self._forward_conv_layers(x)
        # Flatten the tensor to feed into fully connected layers
        x = x.view(x.size(0), -1)
        # Forward pass through the first fully connected layer with ReLU and dropout
        x = self.dropout(F.relu(self.fc1(x)))
        # Forward pass through the second fully connected layer with ReLU
        x = F.relu(self.fc2(x))
        # Output layer
        x = self.fc3(x)
        return x

In [8]:
class Advanced_EEG_MI_CNN_LSTM_Transformer(nn.Module):
    def __init__(self):
        super(Advanced_EEG_MI_CNN_LSTM_Transformer, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=25, out_channels=32, kernel_size=(3, 3), padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # Calculate the size of the input to the LSTM layer
        # Initial scales dimension: 43 -> After three pooling layers: 43 / 8 = 5.375 -> rounded down to 5
        final_scale_dim = 5
        final_time_dim = 15  # Assuming time dimension after pooling layers is 15, adjust if needed
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size=128 * final_scale_dim * final_time_dim, hidden_size=128, num_layers=1, batch_first=True)
        
        # Transformer layer
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model=128, nhead=8)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers=1)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 2)
        
        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)
        
    def _forward_conv_layers(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        return x

    def forward(self, x):
        batch_size = x.size(0)
        # Forward pass through convolutional layers
        x = self._forward_conv_layers(x)
        
        # Flatten for LSTM
        x = x.view(batch_size, -1, 128 * 5 * 15)  # Adjust to match input_size of LSTM
        
        # Forward pass through LSTM
        x, (hn, cn) = self.lstm(x)
        
        # Transformer encoding
        x = self.transformer_encoder(x)
        
        # Take the output from the last sequence step for classification
        x = x[:, -1, :]
        
        # Forward pass through fully connected layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [9]:
# Save the model with date and time in the filename
def save_model(model, path):
    torch.save(model.state_dict(), path)

# Example usage:
model = Advanced_EEG_MI_CNN_LSTM_Transformer()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = correct / total
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

# Save the model with current date and time
current_time = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
model_dir = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\bcic_iv_2a\models"
print("Model directory:", model_dir)
os.makedirs(model_dir, exist_ok=True)
model_filename = os.path.join(model_dir, f'{model.__class__.__name__}_{current_time}.pth')
print("Model filename:", model_filename)
save_model(model, model_filename)
print(f'Model saved as {model_filename}')

c:\Users\chakr\Anaconda3\envs\eeg_thesis\lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/50, Loss: 0.2551
Validation Loss: 0.1333, Validation Accuracy: 0.9706
Epoch 2/50, Loss: 0.1363
Validation Loss: 0.0608, Validation Accuracy: 0.9706
Epoch 3/50, Loss: 0.0527
Validation Loss: 0.0202, Validation Accuracy: 1.0000
Epoch 4/50, Loss: 0.0361
Validation Loss: 0.0103, Validation Accuracy: 1.0000
Epoch 5/50, Loss: 0.0290
Validation Loss: 0.0027, Validation Accuracy: 1.0000
Epoch 6/50, Loss: 0.0157
Validation Loss: 0.0010, Validation Accuracy: 1.0000
Epoch 7/50, Loss: 0.0186
Validation Loss: 0.0003, Validation Accuracy: 1.0000
Epoch 8/50, Loss: 0.0227
Validation Loss: 0.0012, Validation Accuracy: 1.0000
Epoch 9/50, Loss: 0.0245
Validation Loss: 0.0002, Validation Accuracy: 1.0000
Epoch 10/50, Loss: 0.0194
Validation Loss: 0.0009, Validation Accuracy: 1.0000
Epoch 11/50, Loss: 0.0116
Validation Loss: 0.0016, Validation Accuracy: 1.0000
Epoch 12/50, Loss: 0.0179
Validation Loss: 0.0011, Validation Accuracy: 1.0000
Epoch 13/50, Loss: 0.0156
Validation Loss: 0.0009, Validation